# lecture-10 Build a neural network from scractch

> Node:
+ forward: Function, how to calculate the inputs
+ backwards: Function, how to get the gradients when backpropogation
+ gradients: Mapper, the gradient map the this node of its inputs mode
+ inputs: List, the input nodes of this node
+ outputs: List, the output node of this node

In [1]:
import numpy as np

In [2]:
class Node:
    """
    Each node in neual networks will have these attributes and methods
    """
    def __init__(self, inputs=[]):
        """
        if the node is the operator of "ax+b", the input will be x node, and the outputs
        of this is its successors.
        and the value is *ax+b*
        """
        self.inputs = inputs
        self.outputs = []
        self.value = None
        self.gradients = {}
        
        for node in self.inputs:
            node.outputs.append(self)
            
    def forward(self):
        """Forword propogation
        compute the output value based on input nodes and store the value
        into *self.value*
        """
        raise NotImplemented
        
    def backward(self):
        """Backword propogation
        compute the gradient of each input nodes and store the value
        into *self.gradient*
        """
        raise NotImplemented

In [3]:
class Input(Node):
    def __init__(self, name=''):
        self.name = name
        Node.__init__(self, inputs=[])
    
    def forward(self,value=None):
        if value is not None:
            self.value = value
        
    def backward(self):
        self.gradients = {}
        
        for n in self.outputs:
            grad_cost = n.gradients[self]
            self.gradients[self] = grad_cost
            
    def __repr__(self):
        return 'Input Node:{}'.format(self.name)

In [4]:
class Linear(Node):
    def __init__(self, nodes, weights, bias):
        self.w_node = weights
        self.x_node = nodes
        self.b_node = bias
        Node.__init__(self, inputs=[nodes, weights, bias])
        
    def forward(self):
        """compute the wx+b using numpy"""
        self.value = np.dot(self.x_node.value, self.w_node.value) + self.b_node.value
        
    def backward(self):
        for node in self.outputs:
            grad_cost = node.gradients[self]
            
            self.gradients[self.w_node] = np.dot(self.x_node.value.T, grad_cost)
            self.gradients[self.b_node] = np.sum(grad_cost * 1, axis = 0, keepdims=False)
            self.gradients[self.x_node] = np.dot(grad_cost, self.w_node.value.T)

In [5]:
class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])
        self.x_node = node
        
    def _sigmoid(self,x):
        return 1. / (1+np.exp(-1*x))
    
    def forward(self):
        self.value = self._sigmoid(self.x_node.value)
    
    def backward(self):
        y = self.value
        self.partial = y * (1-y)
        
        for n in self.outputs:
            grad_cost = n.gradients[self]
            
            self.gradients[self.x_node] = grad_cost * self.partial

In [6]:
class MSE(Node):
    def __init__(self, y_true, y_hat):
        self.y_true_node = y_true
        self.y_hat_node = y_hat
        Node.__init__(self, inputs=[y_true, y_hat])
        
    def forward(self):
        y_true_flatten = self.y_true_node.value.reshape(-1, 1)
        y_hat_flatten = self.y_hat_node.value.reshape(-1, 1)
        self.diff = y_true_flatten - y_hat_flatten
        self.value = np.mean(self.diff **2)
        
    def backward(self):
        n = self.y_hat_node.value.shape[0]
        self.gradients[self.y_true_node] = (2/n) * self.diff
        self.gradients[self.y_hat_node] = (-2/n) * self.diff

In [7]:
def training_one_batch(topological_sorted_graph):
    # graph top sorted list
    for node in topological_sorted_graph:
        node.forward()
        
    for node in topological_sorted_graph[::-1]:
        node.backward()

In [8]:
def topological_sort(data_with_value):
    feed_dict = data_with_value 
    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outputs:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]
            ## if n is Input Node, set n'value as 
            ## feed_dict[n]
            ## else, n's value is caculate as its
            ## inbounds

        L.append(n)
        for m in n.outputs:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [9]:
def sgd_update(trainable_nodes, learning_rate=1e-2):
    for t in trainable_nodes:
        t.value += -1 * learning_rate * t.gradients[t]

In [10]:
from sklearn.datasets import load_boston

In [11]:
data = load_boston()

In [12]:
X_ = data['data']
y_= data['target']

In [13]:
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

In [14]:
n_features = X_.shape[1]

In [15]:
n_hidden = 10

In [16]:
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)

In [17]:
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Build a graph connection

## 1st build nodes in this graph

In [18]:
X, y = Input(name='X'), Input(name='y')
W1, b1 = Input(name='W1'), Input(name='b1')
W2, b2 = Input(name='W2'), Input(name='b2')

## 2nd build connection relationship

In [19]:
linear_output = Linear(X, W1, b1)
sigmoid_output = Sigmoid(linear_output)
yhat = Linear(sigmoid_output, W2, b2)
loss = MSE(y, yhat)

In [20]:
input_node_with_value = {
    X: X_,
    y: y_,
    W1: W1_,
    W2: W2_,
    b1: b1_,
    b2: b2_
}

In [21]:
graph = topological_sort(input_node_with_value)

In [22]:
graph

[Input Node:y,
 Input Node:W2,
 Input Node:b1,
 Input Node:b2,
 Input Node:W1,
 Input Node:X,

# training

In [26]:
losses=[]
epochs = 1000
batch_size = 16
steps_per_epoch = X_.shape[0]
for i in range(epochs):
    loss = 0
    for batch in range(steps_per_epoch):
        indices = np.random.choice(range(X_.shape[0]), size=10, replace=True)
        X_batch = X_[indices]
        y_batch = y_[indices]
        #X_batch, y_batch = resample(X_, y_, n_samples=batch_size)
        X.value = X_batch
        y.value = y_batch
        training_one_batch(graph)
        learning_rate = 1e-2
        sgd_update(trainable_nodes=[W1,W2,b1,b2], learning_rate=learning_rate)
        
        loss += graph[-1].value
        
    if i % 100 == 0:
        print('Epoch: {}, loss={:.3f}'.format(i, loss/steps_per_epoch))
        losses.append(loss)

Epoch: 0, loss=2.708
Epoch: 100, loss=2.654
Epoch: 200, loss=2.621
Epoch: 300, loss=2.706
Epoch: 400, loss=2.712
Epoch: 500, loss=2.801
Epoch: 600, loss=2.654
Epoch: 700, loss=2.591
Epoch: 800, loss=2.769
Epoch: 900, loss=2.803
